<font size="7">9.7.7 Production Inventory Model

<font size="4">金丸博樹

競争的な企業が、長期的な利潤を最大化する。

企業は期首にsだけの期首製品を持っており、当期にqだけ生産、期末製品としてxだけ残す。
企業はs+q-xを当期に価格pで販売し、価格pはマルコフ過程$p_{t+1} = h(p_t, \epsilon_{t+1})$
に従う。

期首製品sと価格pが与えられたときに、q,xをどの値にするのが利潤を最大化するか？

StateもActionも２変数の場合

In [138]:
using QuantEcon
using BasisMatrices
using Optim
using Plots
plotlyjs( )

Plots.PlotlyJSBackend()

structの作成

In [139]:
struct ProductionInventory
    c::Array{Float64}
    k::Array{Float64}
    pbar::Float64
    rho::Float64
    sigma::Float64
    delta::Float64
    spnodes::Array{Float64}
end

c=[0.5, 0.1];　#生産に関わる費用関数のパラメータ
k=[0.1, 0.1];　#貯蔵に関わる費用関数のパラメータ
pbar=1.0;　#長期均衡価格
rho=0.5;　#価格を決めるパラメータ
sigma=0.2;　#価格を決めるパラメータ
delta=0.9;　#割引率

s, pの範囲を指定。
sをstate1, pをstate2として、二次元の補間を行う。

その後、BasisMatricesで補間。

In [141]:
n = [10; 10]
spmin = [0; 0.5]
spmax = [2; 1.5]
basis =  Basis(SplineParams(n[1]-2, spmin[1], spmax[1]), SplineParams(n[2]-2, spmin[2], spmax[2]))
spnodes, (s_vec, p_vec) = nodes(basis)

Φ = BasisMatrix(basis, Expanded(), spnodes, 0)
initial_c = Φ.vals[1] \ ones(size(Φ.vals[1])[1]);

PI = ProductionInventory(c, k, pbar, rho, sigma, delta, spnodes)

ProductionInventory([0.5, 0.1], [0.1, 0.1], 1.0, 0.5, 0.2, 0.9, [0.0 0.5; 0.0952381 0.5; … ; 1.90476 1.5; 2.0 1.5])

$normal(0, \sigma^2)$
をqnwnormを用いて離散化

In [70]:
nshocks = 5
mu = 0
epsilon, weight = qnwnorm(nshocks, mu, sigma)

([-1.27768, -0.606254, 0.0, 0.606254, 1.27768], [0.0112574, 0.222076, 0.533333, 0.222076, 0.0112574])

ベルマン方程式を作る。

ここで、データ構造は、Value functionが長さ10×10のベクトル, Policy functionが[q, x]の形の長さ2のArrayを成分に持つ、長さ10×10のベクトルである。

生産の費用関数は、$c(q) = c_{1}q+0.5c_{2}q^2$

貯蓄の費用関数は、$k(x) = k_{1}x+0.5k_{2}x^2$

価格は、$p_{t+1} = pbar+\rho(p_t-pbar)+\epsilon$

また、$s_{t+1} = x_t$

In [177]:
typeof([0 0])

Array{Int64,2}

In [188]:
function update_Bellman(PI::ProductionInventory, C::Vector)
    c, k, pbar, rho, sigma, delta, spnodes = PI.c, PI.k, PI.pbar, PI.rho, PI.sigma, PI.delta, PI.spnodes
    V_new = Vector{Float64}(size(spnodes)[1])
    x_opt = Vector{Array}(size(spnodes)[1])

    for i in 1:size(spnodes)[1]
        objective(x:: Vector) = -(spnodes[i, :][2] * (spnodes[i, :][1] + x[1] - x[2]) - (c[1] * x[1] + 0.5 * c[2] * x[1]^2) - (k[1] * x[2] + 0.5 * k[2] * x[2]^2) + delta * dot(weight, [funeval(C, basis, [x[2], pbar + rho * (spnodes[i, :][2] - pbar) + epsilon[k]]) for k in 1:length(epsilon)]))
        opt = optimize(objective, [0, 0], [0, 0], [1, 1], Fminbox{GradientDescent}(); iterations=2)
        V_new[i]= -opt.minimum
        x_opt[i] = opt.minimizer
    end

    C_new = Φ.vals[1] \ V_new
    
    return C_new, x_opt
end


update_Bellman (generic function with 2 methods)

係数の初期値を設定。
iterationの最大回数と、許容誤差を決める。

In [172]:
C = initial_c
tol = sqrt(eps())
max_iter = 1
C_error = 1.0
i = 0;

In [189]:
while C_error > tol && i <= max_iter
    C_computed, x_opt = update_Bellman(PI, C)
    C_error = maximum(abs, C_computed - C)
    copy!(C, C_computed)
    i += 1
end

i, C_error

LoadError: [91mMethodError: no method matching optimize(::#objective#110{Array{Float64,1},Array{Float64,1},Array{Float64,1},Float64,Float64,Float64,Array{Float64,2}}, ::Array{Int64,1}, ::Array{Int64,1}, ::Array{Int64,1}, ::Optim.Fminbox{Optim.GradientDescent}; iterations=2)[0m
Closest candidates are:
  optimize(::Any, ::Any, ::Any, ::AbstractArray, ::Optim.Optimizer) at C:\Users\Hiroki\.julia\v0.6\Optim\src\multivariate/optimize/interface.jl:76[91m got unsupported keyword argument "iterations"[39m
  optimize(::Any, ::Any, ::Any, ::AbstractArray, ::Optim.Optimizer, [91m::Optim.Options[39m) at C:\Users\Hiroki\.julia\v0.6\Optim\src\multivariate/optimize/interface.jl:76[91m got unsupported keyword argument "iterations"[39m
  optimize(::Any, ::Any, ::Any, ::AbstractArray; kwargs...) at C:\Users\Hiroki\.julia\v0.6\Optim\src\multivariate/optimize/interface.jl:56
  ...[39m

In [164]:
gridsize = [10,10]
grid1 = linspace(spmin[1],spmax[1],gridsize[1])
grid2 = linspace(spmin[2],spmax[2],gridsize[2])
grid = gridmake(grid1, grid2)

x = s_vec
y = p_vec
z = reshape(funeval(C, basis, grid), 10, 10)

Plots.surface(x, y, z)

In [163]:
x_vec=zeros(100)
for i in 1:100
    x_vec[i] = x_opt[i][2]
end

x = s_vec
y = p_vec
z = reshape(x_vec, 10, 10)

Plots.surface(x, y, z')